Connected to Python 3.11.9

In [1]:
model="gpt-4o",

In [2]:
from openai import OpenAI
import os

client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))

def ChatGPT_API(prompt):
  # Create a request to the chat completions endpoint
  response = client.chat.completions.create(
    model="gpt-4o",
    # Assign the role and content for the message
    messages=[{"role": "user", "content": prompt}], 
    temperature = 0) # Temperature: Controls the randomness of the response (0 is deterministic, 2 is highly random).
  return response.choices[0].message.content

def extract_features(description):
    # Format the prompt to extract the specified features
    prompt = (
        f"Given the following workout description:\n\n"
        f"{description}\n\n"
        f"Please extract the following features:\n"
        f"1. Repetition - The amount of repetitions in total\n"
        f"2. Time - The time that you should take to do the exercise\n"
        f"3. Format - AMRAP, EMOM, or rounds for time\n"
        f"4. Equipment - Type of equipment example: barbell, dumbbell, squat rack, machine, etc.\n"
        f"5. ExerciseType - The type of exercises performed\n"
        f"6. TargetMuscle - The muscle group targeted as a result of the workout\n"
        f"7. AerobicLevel - high, medium, low\n"
        f"8. Intensity - high, medium, low\n"
        f"9. StabilityDemand - high, medium, low\n"
        f"10. Explosiveness - high, medium, low\n"
        f"11. WeightClass - high, medium, low\n\n"
        f"Provide the extracted features in a structured format."
    )
    
    # Call the ChatGPT API with the formatted prompt
    features = ChatGPT_API(prompt)
    return features

# Example usage
description = "10 dumbbell snatches, 15 burpee box jump-overs, 20 dumbbell snatches, 15 burpee box jump-overs, 30 dumbbell snatches, 15 burpee box jump-overs, 40 dumbbell snatches, 15 burpee box jump-overs, 50 dumbbell snatches, 15 burpee box jump-overs"

features = extract_features(description)
print(features)

Here is the structured format with the extracted features from the given workout description:

1. **Repetition**: 230 repetitions  
   - (10 + 20 + 30 + 40 + 50) dumbbell snatches + (15 * 5) burpee box jump-overs = 230

2. **Time**: Not specified  
   - The workout description does not provide a specific time frame.

3. **Format**: Rounds for time  
   - The workout is structured in rounds with a set number of repetitions.

4. **Equipment**: Dumbbell, Box  
   - The workout involves dumbbells for snatches and a box for burpee box jump-overs.

5. **ExerciseType**: Dumbbell Snatches, Burpee Box Jump-Overs  
   - These are the two types of exercises performed.

6. **TargetMuscle**: Full body  
   - Dumbbell snatches target shoulders, back, and legs, while burpee box jump-overs target the entire body, especially the legs and core.

7. **AerobicLevel**: High  
   - The inclusion of burpees and the overall volume suggests a high aerobic demand.

8. **Intensity**: High  
   - The combination 

In [3]:
from openai import OpenAI
import os
import pandas as pd
import io
import csv

client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))

def ChatGPT_API(prompt):
  # Create a request to the chat completions endpoint
  response = client.chat.completions.create(
    model="gpt-4o",
    # Assign the role and content for the message
    messages=[{"role": "user", "content": prompt}], 
    temperature = 0) # Temperature: Controls the randomness of the response (0 is deterministic, 2 is highly random).
  return response.choices[0].message.content

level_map = {'high':2.5, 'medium': 2,'low':1.5}
weights = [0.3, 0.25, 0.15, 0.2, 0.1]

   
def get_numeric_level(level):
   return level_map[level]

#passed in values here should be high, medium, or low
def get_overall_difficulty(aerobic_level,intensity,stability_demand,explosiveness,weight_class):
  values = [
    get_numeric_level(aerobic_level),
    get_numeric_level(intensity),
    get_numeric_level(stability_demand),
    get_numeric_level(explosiveness),
    get_numeric_level(weight_class)
  ]
  overall_difficulty = sum(v * w for v, w in zip(values, weights))
  if overall_difficulty<2:
    return 'low'
  elif overall_difficulty<2.1:
    return 'medium'
  else:
    return 'high'

In [4]:
def extract_levels(description):
    prompt = (
        f"Given the following crossfit workout description: \n"
        f"{description}\n\n"
        f"Please extract the following features:\n"
        f"1. AerobicLevel - high, medium, low\n"
        f"2. Intensity - high, medium, low\n"
        f"3. StabilityDemand - high, medium, low\n"
        f"4. Explosiveness - high, medium, low\n"
        f"5. WeightClass - high, medium, low\n\n"
        f"Provide the categorizations of the features as a list. Keep in mind that these are crossfit workouts, meaning you have to assess these features based off of their relative difficulty compared to general crossfit workouts, meaning they are comarably SIGNIFICANTLY less intense than you think. In essence, make sure to underestimate." 
        f"For example the list should look something like ['high','medium','low','low','high'] for each respective feature.\n"
        f"Make sure you only output the list of feature and nothing else."
    )

    features = ChatGPT_API(prompt)
    return features


In [5]:
import json
import ast

with open('open_parsed_descriptions.json', 'r') as f:
    descriptions = json.load(f)

descriptions_list = []
levels = []

for key in descriptions:
    descriptions_list.append(descriptions[key]["description"])
    levels.append(ast.literal_eval(extract_levels(descriptions[key]["description"])))



In [6]:
levels

[['medium', 'medium', 'medium', 'medium', 'low'],
 ['medium', 'medium', 'high', 'medium', 'medium'],
 ['medium', 'medium', 'high', 'high', 'medium'],
 ['medium', 'medium', 'high', 'medium', 'medium'],
 ['medium', 'medium', 'medium', 'medium', 'medium'],
 ['medium', 'medium', 'medium', 'medium', 'medium'],
 ['medium', 'medium', 'medium', 'medium', 'medium'],
 ['medium', 'medium', 'medium', 'high', 'medium'],
 ['medium', 'medium', 'high', 'high', 'medium'],
 ['medium', 'medium', 'high', 'low', 'high'],
 ['medium', 'medium', 'medium', 'medium', 'medium'],
 ['medium', 'medium', 'medium', 'medium', 'low'],
 ['medium', 'medium', 'medium', 'medium', 'medium'],
 ['medium', 'medium', 'high', 'low', 'medium'],
 ['medium', 'medium', 'medium', 'high', 'low'],
 ['medium', 'medium', 'medium', 'medium', 'medium'],
 ['medium', 'medium', 'medium', 'high', 'low'],
 ['medium', 'medium', 'medium', 'medium', 'medium'],
 ['medium', 'medium', 'high', 'low', 'high'],
 ['medium', 'medium', 'high', 'high', 'hig

In [7]:
difficulties = []
for i in range(len(levels)):
    difficulties.append(get_overall_difficulty(levels[i][0],levels[i][1],levels[i][2],levels[i][3],levels[i][4]))


In [8]:

def extract_features(description,difficulty):
    # Format the prompt to extract the specified features
    prompt = (
        f"Given the following workout description:\n\n"
        f"{description}\n\n"
        f"Please extract the following features:\n"
        f"1. Repetition - The amount of repetitions in total\n"
        f"2. Time - The time that you should take to do the exercise\n"
        f"3. Format - AMRAP, EMOM, or rounds for time\n"
        f"4. Equipment - Type of equipment example: barbell, dumbbell, squat rack, machine, etc.\n"
        f"5. ExerciseType - The type of exercises performed\n"
        f"6. TargetMuscle - The muscle group targeted as a result of the workout\n"
        f"7. Difficulty - {difficulty}\n"
        f"Provide the extracted features in the description in a csv table, with each column in the table having a different features, and each row in the table representing the category/value. ONLY give me the csv table in your response, nothing else, not even the (`) character or the word (csv)."
    )
    
    # Call the ChatGPT API with the formatted prompt
    features = ChatGPT_API(prompt)
    return features

#adjust current prompt for new interpretation
#output needs additional step to extract the features and "combine" them
#maybe group categories under intensity or have some category variable to store the features
#for now, take the weighted average of the different categories and group them under intensity or smthg else

#break into exercise components or movement components

In [9]:
features = []
for i in range(len(descriptions_list)):
    features.append(extract_features(descriptions_list[i],difficulties[i]))

In [10]:
for thing in features:
    print(thing)

Repetition,Time,Format,Equipment,ExerciseType,TargetMuscle,Difficulty
230,,rounds for time,dumbbell,"dumbbell snatches, burpee box jump-overs","shoulders, legs, core",low
Repetition,Time,Format,Equipment,ExerciseType,TargetMuscle,Difficulty
"As many as possible in 12 minutes",12 minutes,AMRAP,"Dumbbells, Pull-up bar, Barbell","Weighted walking lunge, Toes-to-bars, Power cleans, Bar muscle-ups","Legs, Core, Back, Arms",Medium
Repetition,Time,Format,Equipment,ExerciseType,TargetMuscle,Difficulty
180,24:00,Rounds for Time,Barbell,Chest-to-bar pull-ups; Squat snatches,Back; Shoulders; Legs; Core,High
Repetition,Time,Format,Equipment,ExerciseType,TargetMuscle,Difficulty
220,13 minutes,AMRAP,barbell, medicine ball, rowing machine, Deadlifts, Wall-ball shots, Rowing, Handstand push-ups,Full body,Medium
Repetition,Time,Format,Equipment,ExerciseType,TargetMuscle,Difficulty
440,,Rounds for Time,Barbell,Thrusters,Full Body,Medium
Repetition,Time,Format,Equipment,ExerciseType,TargetMuscle,Difficul

In [11]:
# Use StringIO to simulate a file in memory
features_csv = io.StringIO(features)

# Read the CSV data from the StringIO object
df = pd.read_csv(features_csv)

# Display the first few rows of the DataFrame
print(df.head())

TypeError: initial_value must be str or None, not list